In [0]:
select 
     case when vvcrewcodes.cc is not null then vvcrewcodes.cc else roster.leg_flt end `Flight`,
     legmain.dep `Dep`,  legmain.arr `Arr`,
     case when legmain.std is not null then concat(lpad(floor(legmain.std / 60), 2, '0'), lpad(mod(legmain.std, 60), 2, '0'))
          else concat(lpad(floor(roster.grddutybeg / 60), 2, '0'), lpad(mod(roster.grddutybeg, 60), 2, '0')) end `DepStd`,
     case when legmain.sta is not null then concat(lpad(floor(legmain.sta / 60), 2, '0'), lpad(mod(legmain.sta, 60), 2, '0'))
          else concat(lpad( floor(roster.grddutyend / 60), 2, '0') , lpad(mod(roster.grddutyend, 60), 2, '0')) end `ArrSta`,
     date_format(dateadd(day, leg_day, '1980-01-01'), 'yyyy-MM-dd') `DepDate`,
     crewleg.pax `PAX`, 
     concat(lpad(floor((legmain.std + legmain.ddly) / 60), 2, '0'), lpad(mod((legmain.std + legmain.ddly), 60), 2, '0')) `EstimDep`,
     concat(lpad(floor((legmain.sta + legmain.adly) / 60), 2, '0'), lpad(mod((legmain.sta + legmain.adly), 60), 2, '0')) `ActualArr`,
     legmain.reg `Tail`, 
     vvpositions.code, * 
from data_prod.silver_aims.roster
left join data_prod.silver_aims.legmain on roster.leg_day=legmain.day 
     and roster.leg_dep=legmain.dep 
     and roster.leg_carrier=legmain.carrier 
     and roster.leg_flt=legmain.flt 
     and roster.leg_legcd=legmain.legcd
left join data_prod.silver_aims.legtimes on roster.leg_day=legtimes.day 
     and roster.leg_dep=legtimes.dep 
     and roster.leg_carrier=legtimes.carrier 
     and roster.leg_flt=legtimes.flt 
     and roster.leg_legcd=legtimes.legcd
left join data_prod.silver_aims.vvcrewcodes on roster.dutyno=vvcrewcodes.id
left join data_prod.silver_aims.vvpositions on roster.pos=vvpositions.rank
left join data_prod.silver_aims.vvairctype on roster.ac=vvairctype.actype
left join data_prod.silver_aims.crewleg on roster.dutyno=crewleg.croute
     and roster.leg_day=crewleg.day
     and roster.leg_dep=crewleg.dep
     and roster.leg_carrier=crewleg.carrier
     and roster.leg_flt=crewleg.flt
     and roster.leg_legcd=crewleg.legcd
where roster.id= :crewbid 
and roster.leg_day between (select datediff(day, '1980-01-01', date_sub(:reportdate, 10)) as minday) 
                       and (select datediff(day, '1980-01-01', date_add(:reportdate, 1)) as minday)
order by roster.leg_day, legmain.std asc;

